In [89]:
import pandas as pd

In [90]:
foo = pd.read_csv("Data_by_idtrx.csv")
foo.head()
foo = foo.drop(columns='Unnamed: 0')
# foo = foo.drop(columns='Quantity')
# foo = foo.drop(columns='kode')
indexedData = foo.set_index('tgl_transaksi')
indexedData = indexedData.sort_values('tgl_transaksi', ascending=True)
print(len(indexedData))
indexedData.head()

961112


,no_trx,nama_barang,Quantity,kode
tgl_transaksi,,,,
2016-01-01,413500,snack,1,MN
2016-01-01,413401,es krim,1,MN
2016-01-01,413400,nuget,1,MN
2016-01-01,413399,minuman rasa,1,MN
2016-01-01,413399,obat nyamuk,1,MN


# Memilih Data

In [184]:
mulai = '2016-12-01'                                                                             #mulai (untuk temporal) interval
akhir = '2016-12-31'
data_latih = indexedData.loc[mulai:akhir]

basket = (data_latih[data_latih['kode'] =="MN"].groupby(['no_trx', 'nama_barang'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('no_trx'))
print(len(basket))
total_transaksi = len(basket)


# print(basket.head())
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units) #akhir
# basket_sets

9993


In [185]:
mui_sup = 0.03
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
# Build up the frequent items 

frequent_itemsets = apriori(basket_sets, min_support=mui_sup, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.083358,(air mineral)
1,0.033724,(bumbu)
2,0.035525,(coklat)
3,0.048934,(deterjen)
4,0.072851,(es krim)
5,0.038727,(kopi)
6,0.034224,(kosmetik)
7,0.043430,(kue)
8,0.108176,(mie instan)
9,0.101371,(minuman rasa)


# Membuat Rule

In [186]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift") #, min_threshold=0.5
rules =  rules.drop(['lift', 'leverage','conviction','antecedent support','consequent support'], axis=1)
rules

,antecedents,consequents,support,confidence
0,(mie instan),(snack),0.049735,0.459759
1,(snack),(mie instan),0.049735,0.155507
2,(snack),(minuman rasa),0.042430,0.132666
3,(minuman rasa),(snack),0.042430,0.418559
4,(perabot rt),(snack),0.052537,0.423046
5,(snack),(perabot rt),0.052537,0.164268
6,(perabot rt),(susu),0.031922,0.257051
7,(susu),(perabot rt),0.031922,0.151472
8,(snack),(permen),0.030321,0.094806
9,(permen),(snack),0.030321,0.541071


In [187]:
import copy
conf = 0.28
x = rules[(rules['confidence'] >= conf) ]
x2 = copy.deepcopy(x)
# x = x.drop(['lift', 'leverage','conviction','antecedent support','consequent support'], axis=1)
x2["antecedents"] = x2["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
x2["consequents"] = x2["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
x2
print(len(x))
x

5


,antecedents,consequents,support,confidence
0,(mie instan),(snack),0.049735,0.459759
3,(minuman rasa),(snack),0.042430,0.418559
4,(perabot rt),(snack),0.052537,0.423046
9,(permen),(snack),0.030321,0.541071
11,(susu),(snack),0.087761,0.416429


In [188]:
r = list()
l_t = dict()
ix = 1
for a, b in zip(x2.antecedents, x2.consequents):
    r.append([a,b])
    l_t.update({"rule"+str(ix):[a,b]})
    ix+=1
l_t

{'rule1': ['mie instan', 'snack'],
 'rule2': ['minuman rasa', 'snack'],
 'rule3': ['perabot rt', 'snack'],
 'rule4': ['permen', 'snack'],
 'rule5': ['susu', 'snack']}

In [189]:
x.to_excel("hasil_analisis/"+mulai+"-"+akhir+"_min_sup["+str(mui_sup)+"]_confidence["+str(conf)+'].xlsx')

In [190]:
foo = pd.read_excel("hasil_all_3.xlsx")
tanggal = foo['tanggal'].tolist()
barang = foo['barang'].tolist()

In [191]:
print(len(foo))

429832


In [192]:
ruless = list()
for br, tgl in zip(barang, tanggal):
    rl = list()
    br_ = br.split(',')
    
    for id_rule, rul in l_t.items():
#         print(id_rule)
        if l_t[id_rule][0] in br_ and l_t[id_rule][1] in br_:
            rl.append(id_rule)
        
    ruless.append(rl)
n_tgl = list()
n_rule = list()

for rl, tgl in zip(ruless, tanggal):
    if len(rl)>0:
        n_rule.append(rl)
        n_tgl.append(tgl)

# for rl, tgl in zip(ruless, tanggal):
# #     if len(rl)>0:
#     n_rule.append(rl)
#     n_tgl.append(tgl)

n_tgl2 = list()
n_rule2 = list()
for rl, tgl in zip(ruless, tanggal):
    n_rule2.append(rl)
    n_tgl2.append(tgl)

In [193]:
dictttt ={
    "tanggal":n_tgl,
    "tanggal2":n_tgl,
    "rule":n_rule
}
# df_ = pd.DataFrame.from_dict(dictttt)
dictttt2 ={
    "tanggal":n_tgl2,
    "tanggal2":n_tgl2,
    "rule":n_rule2
}


In [194]:
# dictttt ={
#     "tanggal":n_tgl,
#     "rule":n_rule
# }
# df_ = pd.DataFrame.from_dict(dictttt)
df_ = pd.DataFrame.from_dict(dictttt)
x = df_.sort_values(by='tanggal', ascending=True)
# x.to_excel("hasil_rules.xlxs")
df_2 = pd.DataFrame.from_dict(dictttt2)
x2 = df_2.sort_values(by='tanggal', ascending=True)

In [195]:
rulesx = x.set_index('tanggal')
rulesx = rulesx.sort_values('tanggal', ascending=True)

rulesx2 = x2.set_index('tanggal')
rulesx2 = rulesx2.sort_values('tanggal', ascending=True)

print(len(rulesx))
rulesx2

66122


,tanggal2,rule
tanggal,,
2016-01-01,2016-01-01,[]
2016-01-01,2016-01-01,[]
2016-01-01,2016-01-01,[]
2016-01-01,2016-01-01,[]
2016-01-01,2016-01-01,[]
2016-01-01,2016-01-01,[]
2016-01-01,2016-01-01,[]
2016-01-01,2016-01-01,[]
2016-01-01,2016-01-01,[rule5]


In [204]:
import numpy as np
mulai = '2016-12-01'                                                                             #mulai (untuk temporal) interval
akhir = '2016-12-07'

print(mulai)
print(akhir)
x_loc = rulesx.loc[mulai:akhir]
x_loc2 = rulesx2.loc[mulai:akhir]
total_trx = len(x_loc2)
print(total_trx)
# x_loc

2016-12-01
2016-12-07
2221


In [203]:
rulenya = list()
tanggal = list()
total_transaksi = list()
frequency = list()
minsup = list()

for key, value in l_t.items():
    def akhir(rr):
        r2 = x_loc.rule.tolist()
        urutan = list()
        for ix, i in enumerate(r2):
    #         print(i)
            if rr in i:
                urutan.append(ix)
        return urutan
    rx = key
    test_index = akhir(rx) #menjalankan pengujian rule yang diuji
    panjang = len(test_index)
    lr = "=>".join(l_t[rx])
#     print(lr)
    rulenya.append(lr)
    try:
        mulai = test_index[0]
        akhir = test_index[-1]
        tanggal.append([x_loc.tanggal2.tolist()[mulai],x_loc.tanggal2.tolist()[akhir]])
        total_transaksi.append(total_trx)
        frequency.append(panjang)
        minsup.append(panjang/total_trx)
    except:
        print("tidak ada transaksi")
dictt={
    "tanggal":tanggal,
    "rule":rulenya,
    'tempsupport':minsup,
    "frequency":frequency,
    "total_transaksi":total_transaksi,
}
df_r = pd.DataFrame.from_dict(dictt)
df_r.to_excel("hasil_temporal.xlsx")
df_r

,tanggal,rule,tempsupport,frequency,total_transaksi
0,"[2016-12-01, 2016-12-07]",mie instan=>snack,0.064385,143,2221
1,"[2016-12-01, 2016-12-07]",minuman rasa=>snack,0.038721,86,2221
2,"[2016-12-01, 2016-12-07]",perabot rt=>snack,0.066186,147,2221
3,"[2016-12-01, 2016-12-07]",permen=>snack,0.024764,55,2221
4,"[2016-12-01, 2016-12-07]",susu=>snack,0.095903,213,2221
